<a href="https://colab.research.google.com/github/bob8dod/NLP_project-Review_Analysis/blob/main/Crawling/0.Crawling_11st_all_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Guide**
1. 먼저 <<[중요] 리뷰 데이터 모음 (11st) >>에 serach_kwd의 이름으로 폴더를 생성해주세요!  
    - 해당 폴더에는 각각의 상품의 리뷰를 저장합니다.
2. 그 후 serach_kwd에 검색했던 키워드를 입력하세요!
    - 해당 이름으로 리뷰 통합본이 저장됩니다.
3. 그 이후로는 알아서 하세요!  모르면 연락하시고!

**!! serach_kwd, products만 넣으시면 해당 검색키워드에 해당 하는 각 상품의 리뷰, 상품들의 리뷰 통합본이 공유폴더에 생성된답니다. !!**





In [ ]:
cate_name = '음식'
search_kwd = '영양제' # 검색 키워드
# 상품번호목록
products = {1424252732, 2835570428, 157885070, 89369493, 1374880470, 146482027, 
20594331, 1471956236, 153522374, 18584679, 292240112, 15414741, 1571790585,
29965757, 832274785, 1162818471, 1876215162, 104792525, 1425320437, 
3381397012, 1507992708}

# LET'S GO

## 크롤링(11번가)

In [ ]:
import json
import re
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import time
from multiprocessing import Pool, Manager
import pandas as pd
from tqdm.notebook import tqdm

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
store_path = '...'

data_list = []
for product in tqdm(products):
    url_basic = 'https://www.11st.co.kr/products/{}'.format(product)
    data = requests.get(url_basic, headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser')

    category_path = soup.find('div', attrs={'class':'c_product_category_path'}).find_all('em', attrs={'class':'selected'})
    categories = set()
    for cate in category_path:
        categories.add(cate.text)
    product_name = re.sub('[/]',' ', soup.find('h1', attrs={'class':'title'}).text).strip()
    img_src = soup.find('div', attrs={'class':'img_full'}).find('img')['src']
    price = soup.find('ul', attrs={'class':'price_wrap'}).find('span',attrs={'class':'value'}).text
    review_len = soup.find('strong', attrs={'class':'text_num'}).text
    review_len = int(re.sub('[^0-9]','', review_len))
    print('상품명:', product_name)
    print('카테고리:', categories)
    print('가격:', price)
    print('모든 리뷰 개수:', review_len)

    manager = Manager()
    text = manager.list()
    score = manager.list()

    def Crawling(pageNo):
        try:
            url = 'https://m.11st.co.kr/products/v1/app/products/{}/reviews/list?pageNo={}&sortType=01&pntVals=&rtype=&themeNm='.format(product, pageNo)
            response = urlopen(url)
            json_data = json.load(response)
            for i in range(len(json_data['review']['list'])):
                if json_data['review']['list'][i]['subject']:
                    review = json_data['review']['list'][i]['subject']
                    score = str(json_data['review']['list'][i]['evlPnt'])
                    text.append([score, review])
        except:
            print('오류페이지:', pageNo)

    if __name__ == '__main__':
        start_time = time.time()
        pool = Pool(processes=32)
        pool.map(Crawling,range(review_len//10 + 1))
        pool.close()
        pool.join()
        print('수집한 리뷰 총 개수: ', len(text)) # 빈 리뷰 제외 한 총 리뷰 개수
        print("실행 시간 : %s초" % (time.time() - start_time))

    
    tem_data = pd.DataFrame(list(text), columns=['score','review'])
    print('리뷰 데이터 총 개수:', len(tem_data))
    tem_data.drop_duplicates(['review'], inplace=True)
    tem_data.reset_index(drop=True, inplace=True)
    data_list.append(tem_data)
    print('중복제거 후:', len(tem_data))

    #csv로 저장 -> {이름} 리뷰({리뷰개수}개)
    tem_data.to_csv('{}/{}/{}/{} 리뷰({}개).csv'.format(store_path,cate_name,search_kwd, product_name, len(tem_data)), index=False )
    print('==========================================')

  0%|          | 0/21 [00:00<?, ?it/s]

상품명: [찜15%][락토핏] 생유산균 키즈 베베 패스 1통   아연함유   모유유래
카테고리: {'건강식품', '유산균', '성인 유산균'}
가격: 29,290
모든 리뷰 개수: 5071
수집한 리뷰 총 개수:  3505
실행 시간 : 20.567676782608032초
리뷰 데이터 총 개수: 3505
중복제거 후: 3077
상품명: [신현준 영양제] 꾸미바이트 비타민젤리 모음전(비타민 칼슘 오메가-3 아연)
카테고리: {'식품'}
가격: 9,900
모든 리뷰 개수: 10683
수집한 리뷰 총 개수:  7800
실행 시간 : 43.703131437301636초
리뷰 데이터 총 개수: 7800
중복제거 후: 6479
상품명: [조정석밀크씨슬]GNM 건강한 간 밀크씨슬 실리마린 4박스 (총 4개월분)  루테인 유산균 프로폴리스
카테고리: {'영양제', '건강식품', '밀크씨슬'}
가격: 18,900
모든 리뷰 개수: 22139
오류페이지: 1671
수집한 리뷰 총 개수:  16948
실행 시간 : 86.36787343025208초
리뷰 데이터 총 개수: 16948
중복제거 후: 10115
상품명: 종근당건강 락토핏 5X 생유산균 골드 5통 외 모음전
카테고리: {'건강식품', '유산균', '성인 유산균'}
가격: 70,900
모든 리뷰 개수: 8661
수집한 리뷰 총 개수:  5508
실행 시간 : 34.05051827430725초
리뷰 데이터 총 개수: 5508
중복제거 후: 4612
상품명: 종근당건강 아이케어 오메가3 2병 6개월 혈중중성지질 혈행개선 항산화 눈건강 비타민A 비타민E
카테고리: {'영양제', '건강식품', '오메가3'}
가격: 29,900
모든 리뷰 개수: 20583
수집한 리뷰 총 개수:  19078
실행 시간 : 78.89298820495605초
리뷰 데이터 총 개수: 19078
중복제거 후: 9594
상품명: 종근당 데일리베스트 종합비타민 미네랄 180정*2병 총 12개월분 멀티 영양제 츄잉정 유산균혼합
카테고리: {'건강식품', '멀

In [ ]:
total = pd.concat(data_list, ignore_index=True).reset_index(drop=True)
total.to_csv('{}}/{}/{} 통합 리뷰({}개).csv'.format(store_path,cate_name, search_kwd, len(total)), index=False )
print('총 리뷰 개수:',len(total))
total.head()

총 리뷰 개수: 160529


,score,review
0,5,아이들 꾸준히 먹는데 배변활동도 좋와지고 유산균 좋와용.
1,5,아이랑 같이 매일 잘 먹고있어요 ㅎㅎ
2,5,딸 램이 꺼 손녀딸들 꺼 주.문해서 보내 주었네요 ~~ 모두 건강하길 바라는맘으로요
3,4,손자먹이려고 샀는데 잘먹네요 변색깔이 안좋아서먹이는데 조아졌음좋겠어요
4,5,맨날먹는겁니다. 좋아지는지는 모르겠습니다. 꾸준히 진행중입니다. 1년...
